# Install the requirments for ENV setup

In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1 --progress-bar off
!pip install -qqq transformers==4.34.1 --progress-bar off
!pip install -qqq accelerate==0.23.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq llava-torch==1.1.1 --progress-bar off
!pip install peft
# !pip install gdown

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.2 requires huggingface-hub>=0.21.2, but you have huggingface-hub 0.17.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.1 requires httpx>=0.25.0, but you have httpx 0.24.0 which is incompatible.
jupyterlab 4.2.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kaggle-environments 1.14.11 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires 

# load the lib and modules required for test run of LLMs

In [2]:
import textwrap
from io import BytesIO

import requests
import torch
from llava.constants import DEFAULT_IMAGE_TOKEN, IMAGE_TOKEN_INDEX
from llava.conversation import SeparatorStyle, conv_templates
from llava.mm_utils import (
    KeywordsStoppingCriteria,
    get_model_name_from_path,
    process_images,
    tokenizer_image_token,
)
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from PIL import Image
import matplotlib.pyplot as plt
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import numpy as np
import gc
import pandas as pd
import torch.nn.functional as F
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    balanced_accuracy_score,
    matthews_corrcoef,
    confusion_matrix,
    classification_report,
    roc_auc_score,
    average_precision_score
)
from sklearn.preprocessing import label_binarize

[2025-06-05 00:35:41,883] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2025-06-05 00:35:45.099329: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-05 00:35:45.099449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-05 00:35:45.235796: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
disable_torch_init()
MODEL = "microsoft/llava-med-v1.5-mistral-7b"
model_name = get_model_name_from_path(MODEL)
model_name

'llava-med-v1.5-mistral-7b'

# Load the test images for testing the model

In [4]:
tokenizer, model, image_processor, context_len = load_pretrained_model(
                                                    model_path=MODEL, 
                                                    model_base=None, 
                                                    model_name=model_name, 
                                                    load_8bit=True )

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


You are using a model of type llava_mistral to instantiate a model of type llava. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlavaLlamaForCausalLM were not initialized from the model checkpoint at microsoft/llava-med-v1.5-mistral-7b and are newly initialized: ['model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.17.self

In [5]:
# Load the saved dataset
data = torch.load('/kaggle/input/peft-thyroid-dataset/augmented_dataset.pt', weights_only=False)

# Print sizes
print("Train images:", data['train_images'].shape)
print("Train labels:", len(data['train_labels']))
print("Train IDs:", len(data['train_ids']))  # list of strings

print("Val images:", data['val_images'].shape)
print("Val labels:", len(data['val_labels']))
print("Val IDs:", len(data['val_ids']))

print("Test images:", data['test_images'].shape)
print("Test labels:", len(data['test_labels']))
print("Test IDs:", len(data['test_ids']))

Train images: torch.Size([1035, 3, 360, 560])
Train labels: 1035
Train IDs: 1035
Val images: (45, 3, 360, 560)
Val labels: 45
Val IDs: 45
Test images: (45, 3, 360, 560)
Test labels: 45
Test IDs: 45


In [6]:
for name, _ in model.named_modules():
    if any(x in name for x in target_modules):
        print(name)

NameError: name 'target_modules' is not defined

In [7]:
# gc.collect()
# torch.cuda.empty_cache()
# model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
target_modules = [
    "q_proj", "k_proj", "v_proj", "out_proj",  # from self-attn in vision encoder
    "fc1", "fc2",                              # from MLP in vision encoder
    "mm_projector.0", "mm_projector.2"         # optional: projector layers if used
]

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"  # classification
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 8,364,032 || all params: 7,574,583,296 || trainable%: 0.1104223384066143


In [12]:
from torch.utils.data import Dataset, DataLoader

def map_label_to_tirads(label):
    if "2" in label: return "TI-RADS 2"
    elif "3" in label: return "TI-RADS 3"
    elif "4" in label: return "TI-RADS 4"
    elif "5" in label: return "TI-RADS 5"
    else: return "Unknown"

class ThyroidDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images

        label_map = {"TI-RADS 2": 0, "TI-RADS 3": 1, "TI-RADS 4": 2, "TI-RADS 5": 3}
        self.labels = []

        for label in labels:
            if not isinstance(label, str):
                raise ValueError(f"Unexpected non-string label: {label}")
            coarse_label = map_label_to_tirads(label)
            if coarse_label not in label_map:
                raise ValueError(f"🚨 Unknown label: {label} → {coarse_label}")
            self.labels.append(label_map[coarse_label])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'pixel_values': self.images[idx],  # tensor
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


# Create Datasets
train_dataset = ThyroidDataset(data['train_images'], data['train_labels'])
val_dataset = ThyroidDataset(data['val_images'], data['val_labels'])

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1)

In [9]:
device="cuda"

In [10]:
if not hasattr(model, "classifier"):
    from torch import nn
    model.classifier = nn.Linear(model.config.hidden_size, 4).to(device)
    nn.init.xavier_uniform_(model.classifier.weight)
    nn.init.zeros_(model.classifier.bias)

In [13]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()


In [14]:
from tqdm import tqdm
from torch.nn.functional import cross_entropy

class_labels = ["TI-RADS 2", "TI-RADS 3", "TI-RADS 4", "TI-RADS 5"]
label_to_index = {label: i for i, label in enumerate(class_labels)}

model.to(device)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(20):
    total_loss = 0
    for batch in tqdm(val_loader):
        optimizer.zero_grad()

        # Process image
        processed = image_processor(images=batch['pixel_values'], return_tensors="pt").to(device)
        pixel_values = processed["pixel_values"]

        batch_size = processed["pixel_values"].size(0)
        prompts = ["The TI-RADS classification of the thyroid ultrasound image is:"] * batch_size
        inputs = tokenizer(prompts, return_tensors="pt", padding='longest', truncation=True).to(device)
        

        # Forward pass
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            images=pixel_values,
            output_hidden_states=True,
            return_dict=True
        )
        # print(batch['labels'])
        true_labels = batch['labels'].to(device)  # Ensure labels are 0–3
        # print("Labels:", true_labels)
        # print("Min label:", true_labels.min().item(), "Max label:", true_labels.max().item())
        assert torch.all((true_labels >= 0) & (true_labels < 4)), "🚨 Invalid label detected!"

        hidden = outputs.hidden_states[-1][:, -1, :]
        logits = model.classifier(hidden)  # shape: [B, 4]
        loss = cross_entropy(logits, true_labels)
        # print("Logits stats — mean:", logits.mean().item(), "max:", logits.max().item())
        # print("Loss:", loss.item())
        if not torch.isfinite(loss):
            print(f"🚨 Non-finite loss at step {step}, skipping.")
            continue
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")

100%|██████████| 45/45 [01:04<00:00,  1.43s/it]


Epoch 1: Loss = 292.8392


100%|██████████| 45/45 [01:04<00:00,  1.43s/it]


Epoch 2: Loss = 210.6822


100%|██████████| 45/45 [01:03<00:00,  1.40s/it]


...

Epoch 18: Loss = 107.8121


100%|██████████| 45/45 [00:58<00:00,  1.29s/it]


Epoch 19: Loss = 107.0827


100%|██████████| 45/45 [00:57<00:00,  1.29s/it]

Epoch 20: Loss = 105.6571


In [ ]:
# Save only the LoRA adapter
model.save_pretrained("llava-med-lora-train3")
tokenizer.save_pretrained("llava-med-lora-train3")

In [ ]:
def process_image(image):
    args = {"image_aspect_ratio": "pad"}
    image_tensor = process_images([image], image_processor, args)
    return image_tensor.to(model.device, dtype=torch.float16)

In [ ]:
def create_prompt(prompt: str):
    CONV_MODE = 'llava_v0'
    conv = conv_templates[CONV_MODE].copy()
    roles = conv.roles
    prompt = DEFAULT_IMAGE_TOKEN + "\n" + prompt
    conv.append_message(roles[0], prompt)
    conv.append_message(roles[1], None)
    return conv.get_prompt(), conv

In [ ]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    balanced_accuracy_score, matthews_corrcoef, confusion_matrix,
    classification_report
)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score, average_precision_score

def get_report(gt_labels, predicted_labels, all_probs, report_name):
    print("Report:", report_name)
    true_labels = gt_labels
    pred_labels = predicted_labels
    # Optional: sort class labels explicitly
    classes = sorted(set(true_labels + pred_labels))  # [2, 3, 4]
    
    # Binarize true labels
    y_true_bin = label_binarize(gt_labels, classes=class_labels)
    y_prob = np.array(all_probs)
    
    # Compute metrics
    roc_auc = roc_auc_score(y_true_bin, y_prob, average='macro')
    pr_auc = average_precision_score(y_true_bin, y_prob, average='macro')
    
    # Compute standard multi-class metrics
    metrics = {
        "Accuracy": accuracy_score(true_labels, pred_labels),
        "Macro Precision": precision_score(true_labels, pred_labels, average='macro', zero_division=0),
        "Macro Recall": recall_score(true_labels, pred_labels, average='macro', zero_division=0),
        "Macro F1 Score": f1_score(true_labels, pred_labels, average='macro', zero_division=0),
        "Weighted F1 Score": f1_score(true_labels, pred_labels, average='weighted', zero_division=0),
        "Balanced Accuracy": balanced_accuracy_score(true_labels, pred_labels),
        "Matthews Correlation Coefficient": matthews_corrcoef(true_labels, pred_labels),
        "AUC-ROC (macro)": roc_auc,
        "AUC-PR (macro": pr_auc
    }
    
    # Display metrics summary table
    metrics_df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Score'])
    print("=== Multi-class Summary Metrics ===")
    print(metrics_df.round(4))
    
    # Optional: show full classification report
    print("\n=== Per-class Report ===")
    print(classification_report(true_labels, pred_labels, digits=4))
    print("\n========================")

In [ ]:
import torch.nn.functional as F

def classify_image_by_logits(image: Image, class_labels: list[str], prompt: str):
    image_tensor = process_image(image)
    prompt, conv = create_prompt(prompt)
    
    input_ids = tokenizer_image_token(
        prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt"
    ).unsqueeze(0).to(model.device)

    with torch.inference_mode():
        outputs = model(
            input_ids=input_ids,
            images=image_tensor,
            return_dict=True
        )

    # Get the logits for the next token prediction (only last token in prompt)
    next_token_logits = outputs.logits[0, -1]  # shape: [vocab_size]

    # Compute logits for all class labels
    label_scores = []
    for label in class_labels:
        token_ids = tokenizer(label, add_special_tokens=False)["input_ids"]
        logit_sum = sum([next_token_logits[token_id].item() for token_id in token_ids])
        label_scores.append(logit_sum)

    # Convert logits to probabilities
    label_scores_tensor = torch.tensor(label_scores)
    softmax_probs = F.softmax(label_scores_tensor, dim=0).tolist()

    # Get predicted label
    best_index = int(torch.argmax(label_scores_tensor))
    predicted_label = class_labels[best_index]

    return predicted_label, softmax_probs

def map_label_to_tirads(label):
    if "2" in label: return "TI-RADS 2"
    elif "3" in label: return "TI-RADS 3"
    elif "4" in label: return "TI-RADS 4"
    elif "5" in label: return "TI-RADS 5"
    else: return "Unknown"
    
custom_prompt = "The TI-RADS classification of the thyroid ultrasound image is: "
class_labels = ["TI-RADS 2", "TI-RADS 3", "TI-RADS 4", "TI-RADS 5"]

print(len(data['test_labels']))
gt_labels = []
predicted_labels = []
all_probs = []
for i in range(len(data['test_labels'])):
    img_tensor = data['test_images'][i]  # shape: (3, 360, 560)

    gt_label = map_label_to_tirads(data['test_labels'][i])
    gt_labels.append(gt_label)

    pred, probs = classify_image_by_logits(img_tensor, class_labels, custom_prompt)
    predicted_labels.append(pred)

    all_probs.append(probs)
    print(f"PatientId: {data['test_ids'][i]}, Label:{gt_label}, Predicted: {pred}")

baseline_report = get_report(gt_labels, predicted_labels, all_probs, "Lora Test Data Report")

In [ ]:
custom_prompt = "The TI-RADS classification of the thyroid ultrasound image is: "
class_labels = ["TI-RADS 2", "TI-RADS 3", "TI-RADS 4", "TI-RADS 5"]

print(len(data['val_labels']))
gt_labels = []
predicted_labels = []
all_probs = []
for i in range(len(data['val_labels'])):
    img_tensor = data['val_images'][i]  # shape: (3, 360, 560)

    gt_label = map_label_to_tirads(data['val_labels'][i])
    gt_labels.append(gt_label)

    pred, probs = classify_image_by_logits(img_tensor, class_labels, custom_prompt)
    predicted_labels.append(pred)

    all_probs.append(probs)
    print(f"PatientId: {data['val_ids'][i]}, Label:{gt_label}, Predicted: {pred}")
baseline_report = get_report(gt_labels, predicted_labels, all_probs, "LoRA Val Data Report")